In [2]:
# importing necessary libraries
import numpy as np
import pandas as pd
import warnings
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')

In [3]:
#Reading the data file (anime.csv)
anime = pd.read_csv("Data/anime.csv")
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
# only select tv show and movie
print(anime.shape)
anime = anime[(anime['type'] == 'TV') | (anime['type'] == 'Movie')]
print(anime.shape)

(12294, 7)
(6135, 7)


In [5]:
# only select top anime,with 75% percentile
m = anime['members'].quantile(0.75)
anime = anime[(anime['members'] >= m)]
anime.shape

(1534, 7)

In [6]:
#Reading the data file (rating.csv)
rating = pd.read_csv("Data/rating.csv")
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [7]:
# Replacing -1 rating with not available (NaN)
rating.loc[rating.rating == -1, 'rating'] = np.NaN
rating.head()

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN


In [8]:
# creating index for name of anime
anime_index = pd.Series(anime.index, index=anime.name)
anime_index.head()

name
Kimi no Na wa.                      0
Fullmetal Alchemist: Brotherhood    1
Gintama°                            2
Steins;Gate                         3
Gintama&#039;                       4
dtype: int64

In [9]:
# Inner joining data
joined = anime.merge(rating, how='inner', on='anime_id')
joined.head()

,anime_id,name,genre,type,episodes,rating_x,members,user_id,rating_y
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,99,5.0
1,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,152,10.0
2,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,244,10.0
3,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,271,10.0
4,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,278,NaN


In [10]:
# Creating pivot table
joined = joined[['user_id', 'name', 'rating_y']]

# memory error, so we only use 10000 users
joined = joined[(joined['user_id'] <= 10000)]

pivot = pd.pivot_table(joined, index='user_id', columns='name', values='rating_y')
pivot.head()

name,&quot;Bungaku Shoujo&quot; Movie,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,07-Ghost,11eyes,30-sai no Hoken Taiiku,91 Days,A-Channel,AKB0048,...,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,ef: A Tale of Melodies.,ef: A Tale of Memories.,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
pivot.shape

(9412, 1490)

In [12]:
# Removing users that havent rated any anime
pivot.dropna(axis=0, how='all', inplace=True)
pivot.head()

name,&quot;Bungaku Shoujo&quot; Movie,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,07-Ghost,11eyes,30-sai no Hoken Taiiku,91 Days,A-Channel,AKB0048,...,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,ef: A Tale of Melodies.,ef: A Tale of Memories.,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,7.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
pivot.shape

(9412, 1490)

In [14]:
# Centering the mean around 0 (centered cosine / pearson)
pivot_norm = pivot.apply(lambda x: x - np.nanmean(x), axis=1)
pivot_norm.head()

name,&quot;Bungaku Shoujo&quot; Movie,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,07-Ghost,11eyes,30-sai no Hoken Taiiku,91 Days,A-Channel,AKB0048,...,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,ef: A Tale of Melodies.,ef: A Tale of Memories.,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.646341,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.62614,NaN,NaN,NaN,NaN,NaN,-2.37386,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# fill NaN data with 0
pivot_norm.fillna(0, inplace=True)
pivot_norm.head()

name,&quot;Bungaku Shoujo&quot; Movie,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,07-Ghost,11eyes,30-sai no Hoken Taiiku,91 Days,A-Channel,AKB0048,...,Zoku Natsume Yuujinchou,Zoku Sayonara Zetsubou Sensei,Zombie-Loan,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou Iinkai,ef: A Tale of Melodies.,ef: A Tale of Memories.,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,-0.646341,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.62614,0.000000,0.0,0.0,0.0,0.0,-2.37386,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0


In [16]:
# converting into dataframe to make it easier
user_sim_df = pd.DataFrame(cosine_similarity(pivot_norm, pivot_norm), index=pivot_norm.index, columns=pivot_norm.index)
user_sim_df.head()

user_id,1,2,3,5,7,8,9,10,11,12,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,1.000000,0.070572,0.163512,0.021489,0.0,0.121465,0.080198,0.101994,...,-0.105192,-0.142944,0.007926,0.060427,0.053177,0.027009,-0.012482,0.075733,0.033757,0.068755
5,0.0,0.0,0.070572,1.000000,0.076649,0.014843,0.0,0.031263,0.028026,0.092469,...,0.006000,-0.050402,0.048196,0.013640,0.110812,0.171330,-0.018450,0.050163,0.024021,0.008197
7,0.0,0.0,0.163512,0.076649,1.000000,-0.012930,0.0,0.023659,-0.021080,0.062316,...,0.000000,0.000000,-0.000259,0.026777,0.080782,0.094667,0.017602,0.122164,0.047015,-0.012112


In [17]:
# Function for calcuating Similarity index/score of the users
def get_similar_user(user_id):
    if user_id not in pivot_norm.index:
        return None, None
    else:
        sim_users = user_sim_df.sort_values(by=user_id, ascending=False).index[1:]
        sim_score = user_sim_df.sort_values(by=user_id, ascending=False).loc[:, user_id].tolist()[1:]
        return sim_users, sim_score

In [18]:
# Displaying Similarity index/score
users, score = get_similar_user(3)
for x,y in zip(users[:10], score[:10]):
    print("User {} with similarity of {}".format(x, y))

User 2986 with similarity of 0.3502463920298145
User 3681 with similarity of 0.3407187927372403
User 3028 with similarity of 0.33997937684120616
User 2411 with similarity of 0.33778981442850053
User 4481 with similarity of 0.3376927737260321
User 1966 with similarity of 0.3360271004231097
User 2038 with similarity of 0.3202469495714798
User 1606 with similarity of 0.3194861993583852
User 656 with similarity of 0.31929123836055745
User 3990 with similarity of 0.31674233937417606


In [19]:
# Function for getting anime recommnedation based on similarity index/score
def get_recommendation(user_id, n_anime=10):
    users, scores = get_similar_user(user_id)
    
    # there is no information for this user
    if users is None or score is None:
        return None
    
    # only take 10 nearest users
    user_arr = np.array([x for x in users[:10]])
    sim_arr = np.array([x for x in scores[:10]])
    predicted_rating = np.array([])
    
    for anime_name in pivot_norm.columns:
        filtering = pivot_norm[anime_name].loc[user_arr] != 0.0  
        temp = np.dot(pivot[anime_name].loc[user_arr[filtering]], sim_arr[filtering]) / np.sum(sim_arr[filtering])
        predicted_rating = np.append(predicted_rating, temp)
    
    # don't recommend something that user has already rated
    temp = pd.DataFrame({'predicted':predicted_rating, 'name':pivot_norm.columns})
    filtering = (pivot_norm.loc[user_id] == 0.0)
    temp = temp.loc[filtering.values].sort_values(by='predicted', ascending=False)

    # recommend n_anime anime
    return anime.loc[anime_index.loc[temp.name[:n_anime]]]

In [67]:
print(" Top 10 Anime Recommendation: ")
get_recommendation(3)

 Top 10 Anime Recommendation: 


,anime_id,name,genre,type,episodes,rating,members
1304,3712,Zero no Tsukaima: Princesses no Rondo,"Action, Adventure, Comedy, Ecchi, Fantasy, Har...",TV,12,7.60,210891
3788,10578,C³,"Action, Comedy, Ecchi, School, Supernatural",TV,12,6.88,100425
346,223,Dragon Ball,"Adventure, Comedy, Fantasy, Martial Arts, Shou...",TV,153,8.16,316102
100,30230,Diamond no Ace: Second Season,"Comedy, School, Shounen, Sports",TV,51,8.50,38531
254,18689,Diamond no Ace,"Comedy, School, Shounen, Sports",TV,75,8.25,81384
1316,20541,Mikakunin de Shinkoukei,"Comedy, Romance, School, Slice of Life",TV,12,7.59,133385
38,19,Monster,"Drama, Horror, Mystery, Police, Psychological,...",TV,74,8.72,247562
28,457,Mushishi,"Adventure, Fantasy, Historical, Mystery, Seine...",TV,26,8.78,300030
17,24701,Mushishi Zoku Shou 2nd Season,"Adventure, Fantasy, Historical, Mystery, Seine...",TV,10,8.88,75894
3518,12549,"Dakara Boku wa, H ga Dekinai.","Comedy, Ecchi, Romance, Supernatural",TV,12,6.96,169615


In [21]:
print(" Top 10 Anime Recommendation: ")
get_recommendation(27)

 Top 10 Anime Recommendation: 


,anime_id,name,genre,type,episodes,rating,members
1100,22147,Amagi Brilliant Park,"Comedy, Magic",TV,13,7.68,217003
345,1363,Detective Conan Movie 04: Captured in Her Eyes,"Adventure, Comedy, Mystery, Police, Shounen",Movie,1,8.16,25461
137,1365,Detective Conan Movie 06: The Phantom of Baker...,"Adventure, Mystery, Police, Shounen",Movie,1,8.42,28462
274,1367,Detective Conan Movie 08: Magician of the Silv...,"Adventure, Comedy, Mystery, Police, Shounen",Movie,1,8.23,23130
1800,18277,Strike the Blood,"Action, Ecchi, Fantasy, Harem, School, Shounen...",TV,24,7.44,231387
986,355,Shakugan no Shana,"Action, Drama, Fantasy, Romance, School, Super...",TV,24,7.74,297058
597,2104,Seto no Hanayome,"Comedy, Parody, Romance, School",TV,26,7.95,165008
1044,8675,Seitokai Yakuindomo,"Comedy, School, Shounen, Slice of Life",TV,13,7.71,217617
2480,11785,Haiyore! Nyaruko-san,"Comedy, Parody, Sci-Fi",TV,12,7.25,154595
1816,15699,Haiyore! Nyaruko-san W,"Comedy, Parody, Sci-Fi",TV,12,7.43,84442


In [71]:
print(" Top 10 Anime Recommendation: ")
get_recommendation(97)

 Top 10 Anime Recommendation: 


,anime_id,name,genre,type,episodes,rating,members
156,1210,NHK ni Youkoso!,"Comedy, Drama, Psychological, Romance",TV,24,8.40,291228
200,121,Fullmetal Alchemist,"Action, Adventure, Comedy, Drama, Fantasy, Mag...",TV,51,8.33,600384
849,120,Fruits Basket,"Comedy, Drama, Fantasy, Romance, Shoujo, Slice...",TV,26,7.80,242553
15,199,Sen to Chihiro no Kamikakushi,"Adventure, Drama, Supernatural",Movie,1,8.93,466254
86,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,25,8.54,896229
131,4224,Toradora!,"Comedy, Romance, School, Slice of Life",TV,25,8.45,633817
643,9919,Ao no Exorcist,"Action, Demons, Fantasy, Shounen, Supernatural",TV,25,7.92,583823
411,1689,Byousoku 5 Centimeter,"Drama, Romance, Slice of Life",Movie,3,8.10,324035
4628,1894,Yu☆Gi☆Oh!: Hikari no Pyramid,"Adventure, Comedy, Fantasy, Game",Movie,1,6.68,37398
803,154,Shaman King,"Action, Adventure, Comedy, Drama, Shounen, Sup...",TV,64,7.83,169517
